In [31]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import openai
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate
import pinecone
import os

from dotenv import load_dotenv

In [26]:
load_dotenv()

True

In [27]:
pinecone.init(
    api_key = os.getenv("PINECONE_API_KEY"),
    environment = os.getenv("PINECONE_ENV"),
)

In [28]:
pinecone.list_indexes()

['clementine-loka']

In [43]:
index = pinecone.Index("clementine-loka")

In [44]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.01103,
 'namespaces': {'': {'vector_count': 1103}},
 'total_vector_count': 1103}

In [45]:
from langchain.vectorstores import Pinecone

In [47]:
from langchain.embeddings import GPT4AllEmbeddings

embeddings = GPT4AllEmbeddings()
vectorstore = Pinecone.from_existing_index("clementine-loka", embeddings)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [58]:
vectorstore.similarity_search("What is AWS?")[0].page_content#metadata["source"]

'The AWS Key Management Service (AWS KMS) key that Amazon SageMaker uses to encrypt the asynchronous inference output in Amazon S3.\n\nReplacement\n\nSpecifies the configuration for notifications of inference results for asynchronous inference.\n\nAsyncInferenceNotificationConfig\n\nReplacement\n\nThe Amazon S3 location to upload inference responses to.\n\nReplacement'

In [38]:
llm = GPT4All(model="models/gpt4all-falcon-q4_0.gguf")

In [39]:
llm("What is AWS?")

'\nAWS stands for Amazon Web Services. It is a cloud computing platform that provides various services such as compute, storage, database, analytics, and security to help businesses build, deploy, and manage applications in the cloud.'

In [95]:
from langchain.chains import VectorDBQA, RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), return_source_documents=True)

In [96]:
query = "What is SageMaker?"
result = qa({"query": query})

In [97]:
result["result"]

' SageMaker is a managed service offered by Amazon Web Services (AWS) that provides an end-to-end platform for building, deploying, and managing machine learning (ML) workloads on AWS. It includes tools such as the SageMaker Studio, which allows users to create and manage ML projects, and the SageMaker console, which provides a graphical interface for creating and managing ML projects. SageMaker also includes features such as automatic model training, model versioning, and monitoring of ML pipelines.'

In [99]:
result["source_documents"][0].metadata["source"]

'datasets/raw/sagemaker-projects-whatis.md'

In [100]:
query = "What are all AWS regions where SageMaker is available?"

In [101]:
result = qa({"query": query})

In [102]:
result["result"]

' All supported AWS regions except China (Beijing), Asia Pacific (Jakarta), Middle East (UAE), Asia Pacific (Hyderabad), Asia Pacific (Melbourne), AWS GovCloud (US-East), AWS GovCloud (US-West), Europe (Spain), China (Ningxia), Europe (Zurich) Region.'

In [103]:
result["source_documents"][0].metadata["source"]

'datasets/raw/sagemaker-notebook-instance-root-access-check.md'